## Table of contents
* [Background](#Background)
* [Business Problem](#problem)
* [Target Audience](#Audience)
* [Data](#data)
* [Methodology](#methodology)
* [Python Code](#python)
* [Results](#results)
* [Discussion](#discussion)
* [Conclusion](#conclusion)
* [Disclaimer](#disclaimer)


## Background <a name="Background"></a>

**Calgary** is the fifth largest city in Canada and the third most populated city after only Toronto and Montreal with a population of nearly 1.4 million. Calgary is located in southern Alberta and it is famous for being the centre of oil industry of Canada.The city ranked 32 in the Mercer's "2019 Quality of living city ranking" better than some famous cities such as San Francisco, Paris, London and New York

## Business Problem <a name="problem"></a>

The aim of this project is to find the most optimal community to open a new Liquor Store business. The criteria to consider in order to identify the optimal location will be the current number of liquor stores and also the population of age group of 19-64 in each community.  

## Target Audience <a name="Audience"></a>


The result of this project will be interesting for someone who is already in the Liquore Store business and wants to open another branch or those who are new in this business and looking for an ideal community to open their first Liquor Store

## Data <a name="data"></a>

The information regarding community names, their addresses and the population of each community will be collected from the **City of Calgary’s Open Data Portal and Citizen Dashboard**
The Community names and codes will be collected from the following link:
https://data.calgary.ca/Demographics/Census-by-Community-2019/rkfr-buzb

The following source will be used to get the data of the population of age group 20-64 :
https://data.calgary.ca/Demographics/Civic-Census-by-Community-Age-and-Gender/vsk6-ghca

The geographical coordinates of the centroid of each community are available here:
https://data.calgary.ca/Base-Maps/Community-Points/j9ps-fyst

The **Foursquare** API will be used to explore each community to find the number of the Liquor Stores that are currently available within each community


## Methodology <a name="methodology"></a>

The **Socrata Open Data API** will be used to fetch the data from the **City of Calgary's Open Data Portal and Citizenship Dashboard** and conver it to Pandas Dataframe. As a result, we will have three dataframes: the *Census by Community 2019* dataframe will include the community codes and names,the *Census by Community, Age and Gender* will contain the community codes, the population of 10 different age groups in each community and lastly the *Community Point* dataframe will have the geographical coordinate of the centroid of each community.  

First, I will aggregate the *Census by Community, Age and Gender* dataframe into only 3 different age groups namely 0-19, 20-64 and 65+, then add up the population of all the genders in the 20-64 age group. Then, I will drop the non-residential communities. Then I will consolidate the resulted dataframe with the other two and drop all unneccessary columns. The remaining columns will be Community name, Population of 19-64, Latitude and Longitude 

Second, I will use Foursquare API to find all the Liquore Stores within 4 Kilometers of the centroid of each community 

Third, I will use the **Folium library** to visulize the location of the liquor store in City of Calgary  

Forth, I will append my dataframe with a new column which represents the number of Liquore Stores within three kilometrs of the centroid of each community  

Fifth, I will devide the population of 20-64 by the number of the liquor stores in each community and add the result as a new column to our dataframe. The result numbers represent the number of people between 20 and 64 years old per liquor store in each community. Therefore the community corresponding with the biggest number is the most optimal community to open a new Liquor Store



## Python Code <a name="python"></a>

In [2]:
#Importing necessary libraries
import os
import pandas as pd
import numpy as np
!pip install sodapy
from sodapy import Socrata
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
print('libraries imported')

You should consider upgrading via the 'pip install --upgrade pip' command.
libraries imported


In [3]:
# Forming the first Dataframe by first fetching data from https://data.calgary.ca/Demographics/Civic-Census-by-Community-Age-and-Gender/vsk6-ghca
#https://data.calgary.ca/resource/vsk6-ghca.json
domain = 'data.calgary.ca'
identifier1 = 'vsk6-ghca'
!export SODAPY_APPTOKEN=<XMJrPlE0irficSLKuv383Gmt0>
token = os.environ.get("SODAPY_APPTOKEN")
client = Socrata(domain, token)
results = client.get(identifier1, limit =23000)

# Convert to pandas DataFrame
df1 = pd.DataFrame.from_dict(results)
df1.head()

/bin/sh: 1: Syntax error: end of file unexpected


,age_range,code,females,males,other,year
0,0-4,ABB,0,635,NaN,1996
1,5-14,ABB,0,1438,NaN,1996
2,15-19,ABB,0,462,NaN,1996
3,20-24,ABB,0,347,NaN,1996
4,25-34,ABB,0,2842,NaN,1996


In [4]:
#dropping data related to all years except for 2019
df1_2019=df1[df1['year'] == '2019'] 

df1_2019.head()


,age_range,code,females,males,other,year
20280,0-4,ABB,175,191,0,2019
20281,5-14,ABB,419,527,0,2019
20282,15-19,ABB,168,193,0,2019
20283,20-24,ABB,148,185,0,2019
20284,25-34,ABB,428,407,0,2019


As it can be seen from the above dataframe, the population has been broken down to several age ranges. Since we are interested in the age range of 20-65 we will drop all other age ranges outside our interest range

In [5]:
#dropping unnecessory rows related to ages outside our interest group range
df1_t1= df1_2019[df1_2019['age_range'] != '0-4']
df1_t2= df1_t1[df1_t1['age_range'] != '5-14']
df1_t3= df1_t2[df1_t2['age_range'] != '15-19']
df1_t4= df1_t3[df1_t3['age_range'] != '65-74']
df1_t5= df1_t4[df1_t4['age_range'] != '75+']
df1_t5.head()

,age_range,code,females,males,other,year
20283,20-24,ABB,148,185,0,2019
20284,25-34,ABB,428,407,0,2019
20285,35-44,ABB,501,503,0,2019
20286,45-54,ABB,393,399,0,2019
20287,55-64,ABB,395,397,0,2019


The next step is to sum up the population of all genders in each age range and then sum up the population of all age ranges and aggregate it into one group of 20-64 age range per community

In [6]:
df1_modified=df1_t5.drop(['year', 'age_range'], axis =1)
df1_modified[['males', 'females', 'other']] = df1_modified[['males', 'females', 'other']].apply(pd.to_numeric) 
df1_grouped=df1_modified.groupby('code', as_index=False).sum()
df1_grouped['total adults'] = df1_grouped.iloc[:,-3:].sum(axis=1)
df1_totalAdults=df1_grouped.drop(['males', 'females', 'other'], axis =1)
df1_totalAdults.head()

,code,total adults
0,ABB,3756
1,ACA,6750
2,ALB,4692
3,ALT,4629
4,APP,4577


Our first dataframe that contains the community codes and population of the people between 20 and 64 years old is ready. We need the community names so we use the Socrata API again to fetch the "Census by Community" data that includes both community codes and names

In [7]:
#forming the second dataframe by fetching data from https://data.calgary.ca/Demographics/Census-by-Community-2019/rkfr-buzb
# the API endpoint is https://data.calgary.ca/resource/rkfr-buzb.json
identifier2 = 'rkfr-buzb'
results2 = client.get(identifier2, limit =400)
# Convert to pandas DataFrame
df2 = pd.DataFrame.from_dict(results2)
df2.head()

,apartment,apt_na,apt_no_res,apt_occpd,apt_owned,apt_person,apt_uc,apt_vacant,cat_cnt,class,...,srg,town_house,twn_na,twn_no_res,twn_occpd,twn_owned,twn_person,twn_uc,twn_vacant,unknwn_sch
0,1004.0,2.0,8.0,909.0,394.0,1759.0,2.0,83.0,0.0,Residential,...,DEVELOPING,311.0,0.0,2.0,258.0,222.0,660.0,44.0,7.0,907.0
1,520.0,2.0,0.0,453.0,70.0,656.0,1.0,62.0,0.0,Residential,...,BUILT-OUT,24.0,0.0,0.0,18.0,0.0,46.0,2.0,4.0,1104.0
2,219.0,0.0,0.0,0.0,0.0,0.0,219.0,0.0,0.0,Residential,...,DEVELOPING,128.0,0.0,0.0,91.0,88.0,266.0,36.0,1.0,940.0
3,132.0,0.0,0.0,128.0,20.0,172.0,0.0,4.0,0.0,Residential,...,BUILT-OUT,111.0,0.0,0.0,110.0,42.0,299.0,0.0,1.0,790.0
4,117.0,0.0,0.0,104.0,19.0,254.0,0.0,13.0,0.0,Residential,...,BUILT-OUT,424.0,0.0,1.0,392.0,244.0,1143.0,0.0,27.0,727.0


Some of the communities are non-residential and therefore are not of our interest so we drop them. We also drop all unnecessary columns and only keep the columns contain the community codes and names

In [8]:
#droping non-residentials community
df2_residential= df2[df2['class'] == 'Residential'] 
#dropping all unnecessary columns
df2_cleaned = df2_residential[['comm_code','name']]
df2_cleaned.head()

,comm_code,name
0,LEG,LEGACY
1,HPK,HIGHLAND PARK
2,CNS,CORNERSTONE
3,MON,MONTGOMERY
4,TEM,TEMPLE


In [10]:
#renaming the "comm_code" column to "code" to match df1_totalAdults dataframe
df2=df2_cleaned.rename(columns={'comm_code': 'code'})
df2.head()

,code,name
0,LEG,LEGACY
1,HPK,HIGHLAND PARK
2,CNS,CORNERSTONE
3,MON,MONTGOMERY
4,TEM,TEMPLE


Now we merge the two cleaned dataframes based on the common community codes in each dataframe

In [11]:
#merging df1 and df2 based on common community code
df_merged12=pd.merge(df2, df1_totalAdults, on="code", how = 'inner')
df_merged12.head()

,code,name,total adults
0,LEG,LEGACY,4310
1,HPK,HIGHLAND PARK,2736
2,CNS,CORNERSTONE,1831
3,MON,MONTGOMERY,3159
4,TEM,TEMPLE,6762


In order to use the Foursquare API to search for the available Liquor Stores in each community, we need to have latitude and longitude of centroid of each community. Those data are available in the City of Calgary's "Community Points" dataset. So the next step is to use Socrata API to fetch the latitude and longitude of the centroid of each community and convert it to a Pandas dataframe 

In [12]:
#forming the third dataframe by fetching data from https://data.calgary.ca/Base-Maps/Community-Points/j9ps-fyst
# the API endpoint is https://data.calgary.ca/resource/j9ps-fyst.json
identifier3 = 'j9ps-fyst'
results3 = client.get(identifier3, limit =400)
# Convert to pandas DataFrame
df3 = pd.DataFrame.from_dict(results3)
df3.head()

,:@computed_region_4a3i_ccfj,:@computed_region_4b54_tmc4,:@computed_region_dyj4_ed5g,:@computed_region_kxmf_bzkv,:@computed_region_p8tp_5dkv,class,class_code,comm_code,comm_structure,latitude,location,longitude,name,sector,srg
0,1,6,1,53,6,Residential,1,YKV,BUILDING OUT,50.8704034468757,"{'latitude': '50.870403446876', 'longitude': '...",-114.076647581736,YORKVILLE,SOUTH,DEVELOPING
1,3,6,1,227,13,Residential,1,WWO,BUILDING OUT,50.8707242991201,"{'latitude': '50.87072429912', 'longitude': '-...",-114.008636787791,WOLF WILLOW,SOUTH,DEVELOPING
2,1,1,11,114,14,Residential,1,WSP,BUILDING OUT,51.0597323984505,"{'latitude': '51.059732398451', 'longitude': '...",-114.206168318752,WEST SPRINGS,WEST,DEVELOPING
3,1,5,8,216,6,Residential,1,WOO,1980s/1990s,50.9428758824735,"{'latitude': '50.942875882474', 'longitude': '...",-114.106339454003,WOODLANDS,SOUTH,BUILT-OUT
4,1,8,16,144,5,Residential,1,WND,1950s,51.0050404653601,"{'latitude': '51.00504046536', 'longitude': '-...",-114.083549507337,WINDSOR PARK,CENTRE,BUILT-OUT


Next, we drop the non-residential communities and unnecessary columns

In [13]:
#droping non-residentials community
df3_residential= df3[df3['class'] == 'Residential'] 
#dropping all unnecessary columns
df3_cleaned = df3_residential[['comm_code','longitude', 'latitude' ]]
df3_cleaned.head()

,comm_code,longitude,latitude
0,YKV,-114.076647581736,50.8704034468757
1,WWO,-114.008636787791,50.8707242991201
2,WSP,-114.206168318752,51.0597323984505
3,WOO,-114.106339454003,50.9428758824735
4,WND,-114.083549507337,51.0050404653601


Now we merge the above dataframe with the one resulted from merging DF1 and DF2 and then drop the "Code" column

In [14]:
#renaming the "comm_code" column to "code" to match df_merged12 dataframe
df3=df3_cleaned.rename(columns={'comm_code': 'code'})
#merging df_merged12 and df3 based on common community code
df_merged=pd.merge(df_merged12, df3, on="code", how = 'inner')
#we don't need the community code column anymore so we can drop it to have a cleaner dataframe
df=df_merged.drop(['code'], axis=1)
df.head()

,name,total adults,longitude,latitude
0,LEGACY,4310,-114.005999006031,50.8576170329455
1,HIGHLAND PARK,2736,-114.062066898001,51.0880232734306
2,CORNERSTONE,1831,-113.930582035357,51.1573690704188
3,MONTGOMERY,3159,-114.162261375791,51.0710992820493
4,TEMPLE,6762,-113.946767730631,51.0887951436987


It's time to use the Forsquare API. We need to define our credentials and also import some libraries to handle the fetched data

In [15]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'LIDY1YMEGHDKSVJHH2Q0DCLFMPHVJMTUNNFHUSCFRBSYRS5X' # your Foursquare ID
CLIENT_SECRET = '0IDF2GB2ZFGYJICUSEVPNWFIY5W0OLAB0YH5MFJWEVZMRXIZ' # your Foursquare Secret
VERSION = '20200611'
LIMIT = 20
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LIDY1YMEGHDKSVJHH2Q0DCLFMPHVJMTUNNFHUSCFRBSYRS5X
CLIENT_SECRET:0IDF2GB2ZFGYJICUSEVPNWFIY5W0OLAB0YH5MFJWEVZMRXIZ


In [16]:
#importing necessary libraries
import requests
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

Now we search for Liquor Stores within 4 km of the centroid of each community

In [17]:
search_query = 'Liquor'
radius = 4000
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
rowIndex = 0  
for a,b in zip(df.longitude, df.latitude): 
    latitude=float(b)
    longitude=float(a)
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

    # tranform venues into a dataframe
    try:
        result = requests.get(url).json()
        venues = result['response']['venues']
        dataframe = json_normalize(venues)
        filtered_columns = ['name', 'categories'] 
        dataframe_filtered = dataframe.loc[:, filtered_columns]
        # filter the category for each row
        dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
        # clean column names by keeping only last term
        dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
        df4=dataframe_filtered[dataframe_filtered['categories'] == 'Liquor Store']
        number_of_liquore_stores = len(df4.index)
    except:
        number_of_liquore_stores = 0        
    df.loc[rowIndex, 'nStores'] = number_of_liquore_stores
    rowIndex += 1
df.head()

,name,total adults,longitude,latitude,nStores
0,LEGACY,4310,-114.005999006031,50.8576170329455,8.0
1,HIGHLAND PARK,2736,-114.062066898001,51.0880232734306,18.0
2,CORNERSTONE,1831,-113.930582035357,51.1573690704188,3.0
3,MONTGOMERY,3159,-114.162261375791,51.0710992820493,15.0
4,TEMPLE,6762,-113.946767730631,51.0887951436987,15.0


In [18]:
df['adultsPerStore']=df['total adults']/df['nStores']
df.head()

,name,total adults,longitude,latitude,nStores,adultsPerStore
0,LEGACY,4310,-114.005999006031,50.8576170329455,8.0,538.750000
1,HIGHLAND PARK,2736,-114.062066898001,51.0880232734306,18.0,152.000000
2,CORNERSTONE,1831,-113.930582035357,51.1573690704188,3.0,610.333333
3,MONTGOMERY,3159,-114.162261375791,51.0710992820493,15.0,210.600000
4,TEMPLE,6762,-113.946767730631,51.0887951436987,15.0,450.800000


## Results <a name="results"></a>

In [23]:
print(df[df.adultsPerStore == df.adultsPerStore.max()]) 


             name  total adults          longitude          latitude  nStores  \
57  SKYVIEW RANCH          7808  -113.958374584808  51.1561260801993      4.0   

    adultsPerStore  
57          1952.0  


In [35]:
latitude = 51.1561260801993
longitude = -113.958374584808
search_query = 'Liquor'
radius = 4000
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
result = requests.get(url).json()
venues = result['response']['venues']
dataframe = json_normalize(venues)
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng'] 
dataframe_filtered = dataframe.loc[:, filtered_columns]
# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
df4=dataframe_filtered[dataframe_filtered['categories'] == 'Liquor Store']

skyview_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add the liquore Stores as blue circle markers on map
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.Marker(
    location=[lat, lng],
    popup=label,
    icon=folium.Icon(color='blue')
    ).add_to(skyview_map) 
# display map
skyview_map

## Discussion <a name="discussion"></a>

Based on the observations, most of the communities with high adult population with respect to the number of available liquore stores are located in the North East quarter of the City of Calgary with the Skyview Ranch being the most optimal community to open a new liquor store. There are four liquore stores located within 4 kilometers from the centroid of the Skyview Ranch community with the population of 7808 among the age group of 20-64. Community of Cranston located in the south of Calgary is the second most optimal community to open a new liquor store. There are eight liquore stores located within 4 kilometers from the centroid of the Skyview Ranch community with the population of 12257 among the age group of 20-64

## Conclusion <a name="conclusion"></a>

This study analyzed the number of available liquore stores within 4 kilometers of the centroid of each community in the City of Calgary to find an optimal location to open a new liquor store. The population of the people between 20 and 64 years old was found from the 2019 Civic Census results published in the **City of Calgary's Open Data Portal and Citizenship Dashboard**. The socrata API was used to extract the required data drom the Census results and conver it to Pandas dataframes so they can be used in this analysis. Foursquare API was used to search fo the liquore stores and find the number of available liquor stores within 4000 meters from the centroid of each community. The population of 20-64 age group residing in each community then was divided by the number of liquore stores found from the Foursquare API. The result showed that the most optimal community to open a new liquor store is Skyview Ranch followed by the Cranston community. 

## Disclaimer <a name="disclaimer"></a>

This study was solely done as part of the capstone project to complete the **IBM Data Science Professional Certificate** and it is by no mean should be considered as business or financial advise. 